## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## Data Loading & Pre-Processing
- We load the dataset and clean it

In [ ]:
# loads the dataset
df = pd.read_csv("data/raw/SMSSpamCollection", sep="\t", names = ["label", "text"], encoding="latin-1")


#BAsic info of dataset
print(df.haed())
print(df.label.value_counts())

#convert labels to binary
df["num_label"] = df["label"].map[{"ham": 0, "spam": 1}]

#Train & Test Split
x_train, x_test, y_train, y_test = train_test_split(df["text"], df["num_label"], test_size=0.2, random_state=42, stratify=df["num_label"])


#TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words="english", lowercase=True, max_features=5000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

print("TF-IDF shape:", x_train_tfidf.shape)

## Dimension Reduction Using SVD
-

In [1]:
#Reduce to 100 components

svd= TruncatedSVD(n_components=100, n_iter=52)

x_train_svd = svd.fit_transform(x_train_tfidf)
x_test_svd = svd.transform(x_test_tfidf)

print("SVD shape:", x_train_svd.shape)

#Variance plot

plt.figure(figsize=(8,5))
plt.plot(np.cumsum(svd.explained_variance_ratio_), marker="o")
plt.title("Cumulative Explained Variance by SVD Components")
plt.xlabel("Number of Components")
plt.ylabel("Explained Variance Ratio")
plt.grid(True)


NameError: name 'TruncatedSVD' is not defined

## Model Training - Logistic Regression With SVD

### Model Visualization Confusion Matrix

##### How does reducing dimensions affect model accuracy and speed?


-



